<a href="https://colab.research.google.com/github/tqsyed/fc2conv-tf/blob/main/slmdemolstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical
from doc3 import training_doc3
cleaned = re.sub(r'\W+', ' ', training_doc3).lower()
tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

#Collecting some information
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [ ]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
#print(train_targets[0])

(1392, 3)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding, SimpleRNN
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(SimpleRNN(50, return_sequences=True))
model.add(SimpleRNN(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=30,verbose=1)
model.save("mymodel.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              1365      
                                                                 
 simple_rnn (SimpleRNN)      (None, 3, 50)             2700      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 455)               23205     
                                                                 
Total params: 34,870
Trainable params: 34,870
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
44/44 [=============================

In [ ]:
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-1:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

Two of those planes , American Airlines Flight 11 and United Airlines Flight 175 ,
[58, 5, 253, 101, 102, 54, 45, 11, 7, 25, 54, 45, 254] [[ 54  45 254]]
1/1 [==============================] - 0s 305ms/step
Next word suggestion: was


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding, SimpleRNN
#model = load_model("mymodel.h5")

lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
lstm_model.add(LSTM(50,return_sequences=True))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(50,activation='relu'))
lstm_model.add(Dense(vocabulary_size, activation='softmax'))
print(lstm_model.summary())
# compile network
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(train_inputs,train_targets,epochs=500,verbose=1)
lstm_model.save("mymodel.h5")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 3)              1365      
                                                                 
 lstm (LSTM)                 (None, 3, 50)             10800     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 455)               23205     
                                                                 
Total params: 58,120
Trainable params: 58,120
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
44/44 [==========================

In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:

lstm_input_text = input().strip().lower()
lstm_encoded_text = tokenizer.texts_to_sequences([lstm_input_text])[0]
lstm_pad_encoded = pad_sequences([lstm_encoded_text], maxlen=seq_len, truncating='pre')
print(lstm_encoded_text, lstm_pad_encoded)
for i in (lstm_model.predict(lstm_pad_encoded)[0]).argsort()[-1:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

Two of those planes , American Airlines Flight 11 and United Airlines Flight 175 ,
[58, 5, 253, 101, 102, 54, 45, 11, 7, 25, 54, 45, 254] [[ 54  45 254]]
1/1 [==============================] - 1s 1s/step
Next word suggestion: were
